# LLM Zoomcamp 2024 Competition
This competition requires the use of LLMs to solve high school mathematics problems. The task is to develop models that can accurately solve the problems and then the predictions submitted.

The model in this run is Mistral.

In [1]:
# Import required modules.
import os
import re
import pandas as pd
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from scorer import score
from mistralai import Mistral

/home/codespace/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up Mistral API key.
load_dotenv(Path(find_dotenv()))
api_key = os.getenv("MISTRAL_API_KEY")

In [3]:
# Define the model.
llm = "mistral-large-latest"
client = Mistral(api_key=api_key)

In [4]:
# Function to query the LLM.
def get_answer(question):
    instruction = f'{question} Return as a float'
    chat_response = client.chat.complete(
        model = llm,
        messages = [
            {
                "role": "user",
                "content": instruction,
            },
        ]
    )

    return chat_response.choices[0].message.content

In [5]:
def return_answer(text):
    # Use regular expression to find all numbers in the text
    numbers = re.findall(r'-?\d+\.?\d*', text)

    if numbers:
        # Convert the last found number to a float and return it
        return float(numbers[-1])
    else:
        # Return None if no numbers are found
        return None

In [6]:
# Function to prepare the dataset and get answers.
pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [7]:
# Function for processing each row.
def process_row(row):
    problem_id = row['problem_id']
    problem_text = row['problem_text']

    llm_reasoning = get_answer(problem_text)

    numerical_answer = return_answer(llm_reasoning)

    return {
        'problem_id': problem_id,
        'problem_text': problem_text,
        'llm_reasoning': llm_reasoning,
        'answer': numerical_answer
    }

In [8]:
# Load the train data.
df_train = pd.read_csv('data/train.csv')
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [9]:
# Process row.
rows = df_train.head().to_dict(orient='records')
process_row(rows[0])

{'problem_id': 2374,
 'problem_text': 'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.',
 'llm_reasoning': "To find the value of the expression \\(\\frac{17}{5} : \\frac{34}{3} + 1.3\\), let's break it down step by step.\n\n1. **Division of fractions:**\n   The expression \\(\\frac{17}{5} : \\frac{34}{3}\\) can be rewritten as \\(\\frac{17}{5} \\div \\frac{34}{3}\\).\n\n2. **Convert division to multiplication:**\n   When dividing fractions, we multiply by the reciprocal of the divisor:\n   \\[\\frac{17}{5} \\div \\frac{34}{3} = \\frac{17}{5} \\times \\frac{3}{34}\\]\n\n3. **Simplify the multiplication:**\n   \\[\\frac{17}{5} \\times \\frac{3}{34} = \\frac{17 \\times 3}{5 \\times 34} = \\frac{51}{170}\\]\n\n4. **Simplify the fraction:**\n   \\[\\frac{51}{170} = \\frac{3}{10}\\]\n\n5. **Add 1.3 to the result:**\n   Convert 1.3 to a fraction:\n   \\[1.3 = \\frac{13}{10}\\]\n\n   Now add the two fractions:\n   \\[\\frac{3}{10} + \\frac{13}{10} = \\frac{3 + 13}{10} =

In [10]:
# Return results df_train head.
results = map_progress(pool, rows, process_row)
df_results = pd.DataFrame(results)
df_results

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.86s/it]


,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,To find the value of the expression \(\dfrac{1...,1.6
1,4723,"In a company of 30 people, 25 use the social n...","To determine which statements are true, we wil...",24.0
2,7135,The number of road traffic accidents (RTAs) in...,To find the percentage decrease in the number ...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,To find the value of the expression \(\dfrac{2...,256.0
4,9237,A traveler from Moscow wants to visit four cit...,To determine the routes that the traveler shou...,35.0


100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [04:23<00:00, 28.53s/it]

In [11]:
# Show df_train head.
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [12]:
# Score df_train head results.
score(df_train.head(), df_results, 'problem_id')

0.8

In [17]:
# Put all together in a function.
def prepare_prompts_and_get_answers(df):
    rows = df.to_dict(orient='records')
    results = map_progress(pool, rows, process_row)
    return pd.DataFrame(results)

In [18]:
# Return the results head.
df_train_results = prepare_prompts_and_get_answers(df_train)
df_train_results.head()

100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:08<00:00,  1.89s/it]


,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,To find the value of the expression \(\dfrac{1...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",To determine which statements are true under t...,24.0
2,7135,The number of road traffic accidents (RTAs) in...,To find the percentage decrease in the number ...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,To find the value of the expression \(\dfrac{2...,256.0
4,9237,A traveler from Moscow wants to visit four cit...,To determine the optimal set of routes for the...,35.0


In [19]:
# Get the total score.
score(df_train, df_train_results, 'problem_id')

0.75

In [21]:
# Prepare the test submission.
df_test = pd.read_csv('data/test.csv')

df_test_results = prepare_prompts_and_get_answers(df_test)

submission = df_test_results[['problem_id', 'answer']]
submission.to_csv('submission_mistral.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [04:23<00:00,  2.64s/it]
